In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import shuffle
from scipy.integrate import ode, odeint
from scipy.sparse import csgraph
from scipy.signal import find_peaks
from scipy.spatial import distance
from scipy.stats import norm
from scipy.interpolate import interp1d
import pickle
from matplotlib.pylab import rcParams
from matplotlib import animation
from IPython.display import HTML

In [2]:
%matplotlib inline
rcParams['figure.figsize'] = 15, 6
rcParams['animation.html'] = 'html5'

In [3]:
def make_vanderpl(alpha,omega0_square):
    def vanderpol(X, t):
        x = X[0]
        y = X[1]
        dxdt = y
        dydt = alpha * (1 - x**2)*y-omega0_square*x
        return [dxdt, dydt]
    return vanderpol

In [4]:
def consecutive(data, stepsize=1):
    return np.hsplit(data, np.where(np.diff(data) != stepsize)[0]+1)

In [ ]:
def vanDerPolTrajectory(alpha, omega0_square,X0=[2,0],resolution = 2000, t_end = 150):
    t = np.linspace(0,t_end, t_end*resolution+1)
    
    return odeint(make_vanderpl(alpha,omega0_square), X0, t)
    

In [5]:
def TandLC(trajectory, alpha, omega0_square):
    # Neighborhood Radius
    cutoff = np.floor(t_end*resolution*0.2).astype(int)
    T = np.empty(4, dtype=object)
    
    sol_cut = trajectory[-cutoff:]
    peaks, _ = find_peaks(sol_cut[:,0],height=0)
    try:
        ind1= peaks[-2]
        ind2= peaks[-1]
        TrackLC = sol_cut[ind1:ind2+1]
        phaseZ = np.argmax(TrackLC[:,0])
        T[0]= alpha 
        T[1]=omega0_square
        T[2]=((ind2-ind1)/resolution)
        T[3]= np.roll( TrackLC, -phaseZ, axis = 0 )
        return T
        
    except:
        print(alpha)
        print(omega0_square)
        print(peaks)
        T[0]= alpha
        T[1]= omega0_square
        T[2]= np.nan
        T[3]= trajectory#np.nan
        return T

In [7]:
def phaseResponseCurve(LC,
                       alpha=0.005,
                       omega0_square=1,
                       delta = 10**(-1),
                       resolution=2000,
                       testNumber =100,
                      direction = "dx"):
    T=LC.shape[0]
    t = np.linspace(0,10* T/resolution, 10*T+1)
    deltaT= T//testNumber
    t_step= np.arange(0,T,deltaT)
    phase_response = np.zeros(len(t_step))
    for ind, t_start in enumerate(t_step):
        X0_origin=[LC[t_start,0], LC[t_start,1]]
        if (direction=="dx"):
            X0_per = [LC[t_start,0]+delta, LC[t_start,1]]
        else:
            X0_per = [LC[t_start,0], LC[t_start,1]+delta]
        sol_origin = odeint(make_vanderpl(alpha,omega0_square), X0_origin, t)

        sol_per = odeint(make_vanderpl(alpha,omega0_square), X0_per, t)
        
        
        posDif_per=LC-sol_per[-1,:]
        posDif_origin=LC-sol_origin[-1,:]

        t_origin=np.argmin(np.sum(posDif_origin**2,axis=1))
        t_per=np.argmin(np.sum(posDif_per**2,axis=1))
        

        t_diff = -np.float64(t_per-t_origin)/T
        if t_diff >0.9:
            t_diff -= 1
        if t_diff <-0.9:
            t_diff += 1
        phase_response[ind] = t_diff*360
    return phase_response

In [ ]:
def phaseInterpol(x):
    peaks, _ = find_peaks(x,height=1,distance=40)
    Ts =np.diff(peaks)
    head =np.linspace(0,2*np.pi,num= Ts[0])[-peaks[0]-1:-1]
    phase = np.concatenate([np.linspace(0,2*np.pi,num=Tn+1)[:-1] for Tn in Ts])
    if peaks[0] != 0:
        phase = np.concatenate((head,phase ))
    while len(phase) < len(x):
        phase = np.concatenate((phase, np.linspace(0,2*np.pi,num=Ts[-1]) ))[:len(x)]
    return phase

In [19]:
def tableOfLC(alpha, omega0_sqr):
    table = np.empty((len(alpha),len(omega0_sqr),4),dtype=object)
    for idx , alp in enumerate(alpha):
        for idy, ome in enumerate(omega0_sqr):
                    Trajectory = vanDerPolTrajectory(alp,ome,X0,resolution, t_end)
                    table[idx][idy]= TandLC(Trajectory,alp,ome)
    return table

In [9]:
def order_param(theta, N, k=1):
    """Returns kth order parameter
    """
    Rk = np.sum( np.exp(1j * k * theta.reshape(-1,N)), axis=1 )/N
    return np.abs(Rk), np.angle(Rk)

## Sampling with Gaussian distribution

In [8]:
np.random.seed(3) 
mean_w = 0.75                            #mean of distribution of natural frequencies of oscillators
std_w = 0.05                           #standard deviation of disribution of natural freqs
N=500
K = None #will loop over this          #coupling strength
sample = np.random.normal(mean_w, std_w, N)
g0= norm.pdf(0,0,std_w)
Kc=2/(np.pi*g0)
print("Critical Value of Coupling: ",Kc)

Critical Value of Coupling:  0.07978845608028654


## $\alpha =1$

In [14]:
alpha1 = 1
X0 = [2, 0]


omega0SquareResolution = 10
omega0SquareStart = 0
omega0SquareEnd = 16
Talpha1 = np.empty(((omega0SquareEnd-omega0SquareStart)*omega0SquareResolution+1,4),dtype=object)

for idx,omega0Square in enumerate(np.linspace(omega0SquareStart,omega0SquareEnd,(omega0SquareEnd
                                                                                 -omega0SquareStart)*omega0SquareResolution+1)) :
    Trajectory = vanDerPolTrajectory(alpha1, omega0Square,X0,resolution, t_end)
    Talpha1[idx]= TandLC(Trajectory,alpha1,omega0Square)



1
0.0
[]
1
0.1
[46159]


In [15]:
NotNanIdx = np.where(~np.isnan(np.asarray(Talpha1[:,2],dtype=np.float64)))

In [16]:
interpolatorOmegaSquareOmegaialpha1 =  interp1d(2*np.pi/Talpha1[:,2][NotNanIdx], (Talpha1[:,1][NotNanIdx]))

In [17]:
TargetOmega0Square1 = np.asarray(interpolatorOmegaSquareOmegaialpha1(sample),dtype=np.float64)

In [18]:
np.any(np.isnan(TargetOmega0Square1))

False

## coupled van der Pol

In [33]:
G= np.ones((N,N))-np.diag(np.ones(N))
L=csgraph.laplacian(G, normed=False)

### copled with x

In [34]:
def coupled_vanderpol( t,x , K =0,alpha = 30,omega0 = 1):
    n = len(x)//2
    dx0dt = x[n:]-K/n*np.dot(L,x[:n])
    dx1dt = alpha * (1 - x[:n]**2)*x[n:]-omega0*x[:n]
    return np.hstack((dx0dt,dx1dt))

def J_func( t, x , K, alpha, omega0):
    n = len(x)//2
    return np.vstack(
        (np.hstack((-K/n*L, np.diag(np.ones(n)))),
        np.hstack((np.diag(-omega0-2*alpha*x[:n]*x[n:]),np.diag(alpha*(1-x[:n]**2)))))
    )

In [90]:
t0=0
t_end_o =600
dt= 0.1
t_half = int((t_end_o-t0)/dt*0.5)
alpha = alpha1
omega0Sqrare = TargetOmega0Square1

X0=[2]*N+[0]*N
Hist_OP = []
OP=[]
OPTrack = []

In [91]:
solver = ode(coupled_vanderpol, jac=J_func)
solver.set_integrator('dopri5', atol=1e-10, rtol=1e-8,nsteps=100000)

In [1]:
for K in np.hstack((np.linspace(1.2,0.025,48),np.linspace(0,1.175,48))):

    if len(solver.y) != 1 and (solver.y).any !=0 :
        X0 = solver.y
    solver.set_f_params(K,alpha,omega0Sqrare)
    solver.set_initial_value(X0, t0)
    t_end = t_end_o
    points = np.empty((int((t_end-t0)/dt)+1,2*N))
    i = 0
    while solver.successful() and solver.t < t_end:
        point = solver.integrate(solver.t+dt)
        points[i]=point
        i += 1


    pointsnxy = points.reshape(len(points),2,-1)
    xs= pointsnxy[:,0,:].T
    phasesDyn = np.concatenate([[phaseInterpol(test)] for test in xs]).T
    OPs = np.concatenate([order_param(phi,N) for phi in phasesDyn],axis=1)
    OP=OPs[0,:]
    print('K= %8.3f  ,Mean: %3.3f ' ' ,Std: %5.4f , return: %2d'%(K,np.mean(OP[-t_half:]),np.std(OP[-t_half:]),solver.get_return_code()))
    # Hist_OP = np.vstack((Hist_OP,[K,np.mean(OP[:,0]),np.std(OP[:,0])]))
    OPTrack.append(OP)
    if len(Hist_OP) == 0:
        Hist_OP = np.array([[K,np.mean(OP[-t_half:]),np.std(OP[-t_half:])]])
    else:
        Hist_OP = np.vstack((Hist_OP,[K,np.mean(OP[-t_half:]),np.std(OP[-t_half:])]))
        
with open('./data/Hist_OP_alpha%2.3f_x_t%3d.pkl'%(alpha,t_end_o), 'wb') as f:  # Python 3: open(..., 'wb')
        pickle.dump(Hist_OP, f)
        
with open('./data/OPTrack_alpha%2.3f_x_t%3d.pkl'%(alpha,t_end_o), 'wb') as f:  # Python 3: open(..., 'wb')
        pickle.dump(OPTrack, f)

## $\alpha =10$

In [3]:
alpha10 = 10
X0 = [2, 0]


omega0SquareResolution = 10
omega0SquareStart = 0
omega0SquareEnd = 16
Talpha10 = np.empty(((omega0SquareEnd-omega0SquareStart)*omega0SquareResolution+1,4),dtype=object)

for idx,omega0Square in enumerate(np.linspace(omega0SquareStart,omega0SquareEnd,(omega0SquareEnd
                                                                                 -omega0SquareStart)*omega0SquareResolution+1)) :
    Trajectory = vanDerPolTrajectory(alpha10, omega0Square,X0,resolution, t_end)
    Talpha10[idx]= TandLC(Trajectory,alpha10,omega0Square)



In [15]:
NotNanIdx = np.where(~np.isnan(np.asarray(Talpha10[:,2],dtype=np.float64)))

In [16]:
interpolatorOmegaSquareOmegaialpha10 =  interp1d(2*np.pi/Talpha10[:,2][NotNanIdx], (Talpha10[:,1][NotNanIdx]))

In [17]:
TargetOmega0Square10= np.asarray(interpolatorOmegaSquareOmegaialpha10(sample),dtype=np.float64)

In [18]:
np.any(np.isnan(TargetOmega0Square10))

False

In [90]:
t0=0
t_end_o =600
dt= 0.1
t_half = int((t_end_o-t0)/dt*0.5)
alpha = alpha10
omega0Sqrare = TargetOmega0Square10

X0=[2]*N+[0]*N
Hist_OP = []
OP=[]
OPTrack = []

In [91]:
solver = ode(coupled_vanderpol, jac=J_func)
solver.set_integrator('dopri5', atol=1e-10, rtol=1e-8,nsteps=100000)

In [1]:
for K in np.hstack((np.linspace(1.2,0.025,48),np.linspace(0,1.175,48))):

    if len(solver.y) != 1 and (solver.y).any !=0 :
        X0 = solver.y
    solver.set_f_params(K,alpha,omega0Sqrare)
    solver.set_initial_value(X0, t0)
    t_end = t_end_o
    points = np.empty((int((t_end-t0)/dt)+1,2*N))
    i = 0
    while solver.successful() and solver.t < t_end:
        point = solver.integrate(solver.t+dt)
        points[i]=point
        i += 1


    pointsnxy = points.reshape(len(points),2,-1)
    xs= pointsnxy[:,0,:].T
    phasesDyn = np.concatenate([[phaseInterpol(test)] for test in xs]).T
    OPs = np.concatenate([order_param(phi,N) for phi in phasesDyn],axis=1)
    OP=OPs[0,:]
    print('K= %8.3f  ,Mean: %3.3f ' ' ,Std: %5.4f , return: %2d'%(K,np.mean(OP[-t_half:]),np.std(OP[-t_half:]),solver.get_return_code()))
    # Hist_OP = np.vstack((Hist_OP,[K,np.mean(OP[:,0]),np.std(OP[:,0])]))
    OPTrack.append(OP)
    if len(Hist_OP) == 0:
        Hist_OP = np.array([[K,np.mean(OP[-t_half:]),np.std(OP[-t_half:])]])
    else:
        Hist_OP = np.vstack((Hist_OP,[K,np.mean(OP[-t_half:]),np.std(OP[-t_half:])]))
        
with open('./data/Hist_OP_alpha%2.3f_x_t%3d.pkl'%(alpha,t_end_o), 'wb') as f:  # Python 3: open(..., 'wb')
        pickle.dump(Hist_OP, f)
        
with open('./data/OPTrack_alpha%2.3f_x_t%3d.pkl'%(alpha,t_end_o), 'wb') as f:  # Python 3: open(..., 'wb')
        pickle.dump(OPTrack, f)

